## Exploration of using Google Places API for use in mapping out science related businesses that have science related job postings in Houston, Texas. 

#### started 2017-02-17

#### import all needed python modules

In [1]:
import requests
import configparser
import json
import geojson
from geojson import Point, Feature, FeatureCollection
# https://github.com/frewsxcv/python-geojson
import time
import pandas as pd

### using <a href="https://docs.python.org/3/library/configparser.html">configparser</a> to handle config file 

In [2]:
config = configparser.ConfigParser()

In [3]:
configFilePath = './config.conf'
# configparser.ConfigParser().read(configFilePath)
config.read(configFilePath)

['./config.conf']

sets key as variable that contains the google-places-api key

In [4]:
# key = configparser['google-places-api']['key1']
key = config['google-places-api']['key1']

In [5]:
key_pre = '&key='

In [6]:
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query='

*next_page_base_url* string used when a result with the base url and query has more than 20 results. Next page includes 20 more results. There is a possiblity for a third list of results, but most of the results with that many locations include many false positives, so only 40 locations were found at max for each search term combination.

In [7]:
next_page_base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?'

# Using big list of companies to create single geoJSON

In [8]:
# using pandas as pd. Pandas should already be imported
df=pd.read_csv('../DATA/biglist_2d_resultsAll_Handedited.csv', sep=',',header=None)
company_array = df.values

len(company_array)

1917

In [10]:
company_array

array([['2nd.md'],
       ['3coast'],
       ['3d-Medical-Staffing,-LLC'],
       ..., 
       ['Zephyr-Environmental-Corporation'],
       ['Zilmed-1'],
       ['Zimmerman-Medical-Clinic']], dtype=object)

#### reformatting the array to be of strings only

In [45]:
company_array_formatted = []
for each in company_array:
    each = each[0]
    company_array_formatted.append(each)
    
company_array_formatted

['2nd.md',
 '3coast',
 '3d-Medical-Staffing,-LLC',
 '3d-Medical-Staffing%2C-LLC',
 '3i-International',
 '3M',
 '4th-Down-Solutions',
 'A-Dental-Care-3',
 'Ab-Staffing-Solutions,-LLC',
 'Ab-Staffing-Solutions%2C-LLC',
 'Abb',
 'Abbott-Laboratories',
 'Abilities-Unlimited-Inc.-1',
 'Abm-On--site',
 'Absolute-Physical-Therapy-and-Fitness-1',
 'Academy-Sports',
 'Academy-Sports-%2B-Outdoors',
 'Acadian-Ambulance',
 'Accc-Insurance-Company',
 'Accenture',
 'Access-Staffing-LLC',
 'Accessible-HR-1',
 'Accountable-Healthcare-Staffing',
 'Accounting-Principals',
 'Ace-Clinical-Research',
 'Ace-Parking-Management',
 'Activ8-Group',
 'Actuant-Corporation',
 'Acu-of-Texas',
 'Adams-Consulting-Group',
 'Addison-Group',
 'Adecco',
 'Adecco-USA-b5c07591',
 'Adler-Tank-Rentals',
 'Adroit-Partners%2C-LLC',
 'Advance-Plus-Therapy',
 'Advanced-Search-Group-2',
 'Advantage-Rent-A-Car',
 'Advantage-Solutions',
 'Advantage-Technical',
 'Advisor-Employee-Services',
 'AECOM',
 'Aequor-Technologies',
 'Aescul

In [32]:
# replace some of the characters so the search is a bit better:
for each in company_array_formatted:
    #print(each, "and type is ",type(each))
    #print(each.find("%2C",1))
    #print(type(each.find("%2C",1)))
    if each.find("%2C",1) > 0:
        each = each.replace("%2C",",")
    else:
        continue
    if each.find("%26",1) > 0:
        each = each.replace("%26","&")
    else:
        continue
    #print(each, "again with each")
    if  each.find("-1",1) >0:
        each.replace("-1","")
    else:
        continue
    if  each.find("-2",0) >0:
        each.replace("-2","")
    else: 
        continue
    if  each.find("-3",0) >0:
        each.replace("-3","")
    else:
        continue
    if  each.find("-",0) >0:
        each.replace("-","+")
    else:
        continue

company_array_formatted      

[]

In [94]:
company_array_formatted

['2nd.md',
 '3coast',
 '3d-Medical-Staffing,-LLC',
 '3d-Medical-Staffing%2C-LLC',
 '3i-International',
 '3M',
 '4th-Down-Solutions',
 'A-Dental-Care-3',
 'Ab-Staffing-Solutions,-LLC',
 'Ab-Staffing-Solutions%2C-LLC',
 'Abb',
 'Abbott-Laboratories',
 'Abilities-Unlimited-Inc.-1',
 'Abm-On--site',
 'Absolute-Physical-Therapy-and-Fitness-1',
 'Academy-Sports',
 'Academy-Sports-%2B-Outdoors',
 'Acadian-Ambulance',
 'Accc-Insurance-Company',
 'Accenture',
 'Access-Staffing-LLC',
 'Accessible-HR-1',
 'Accountable-Healthcare-Staffing',
 'Accounting-Principals',
 'Ace-Clinical-Research',
 'Ace-Parking-Management',
 'Activ8-Group',
 'Actuant-Corporation',
 'Acu-of-Texas',
 'Adams-Consulting-Group',
 'Addison-Group',
 'Adecco',
 'Adecco-USA-b5c07591',
 'Adler-Tank-Rentals',
 'Adroit-Partners%2C-LLC',
 'Advance-Plus-Therapy',
 'Advanced-Search-Group-2',
 'Advantage-Rent-A-Car',
 'Advantage-Solutions',
 'Advantage-Technical',
 'Advisor-Employee-Services',
 'AECOM',
 'Aequor-Technologies',
 'Aescul

In [111]:
# replace some of the characters so the search is a bit better:
def cleanCompanies(company_array):
    new_array = []
    for each in company_array:
        #print(each, "and type is ",type(each))
        print(each.find("-",0))
        print(type(each.find("-",0)),"type of each.find('-',0")
        #print(type(each.find("%2C",1)))
        if each.find("%2C",1) is not -1:
            each = each.replace("%2C",",")
            print(each,"= each")
        else:
            each = each
        if each.find("%26",1) is not -1:
            each = each.replace("%26","&")
            #print(each,"= each find")
        else:
            each = each
        ##print(each, "again with each")
        if  each.find("-1",1) is not -1:
            each = each.replace("-1","")
        #    print(each,"= each find")
        else:
            each = each
        if  each.find("-2",0) is not -1:
            each = each.replace("-2","")
            print(each,"= each find")
        else: 
            each = each
        if  each.find("-3",0) is not -1:
            each = each.replace("-3","")
        #    print(each,"= each find")
        else:
            each
        if  each.find("-",1) is not -1:
            each = each.replace("-","+")
            print(each,"= each find")
        else:
            each = each
        print("each at end",each)
        new_array.extend([each])
    print('new_array',new_array)
    return new_array

In [112]:
company_array_formattedB = cleanCompanies(company_array_formatted)

-1
<class 'int'> type of each.find('-',0
each at end 2nd.md
-1
<class 'int'> type of each.find('-',0
each at end 3coast
2
<class 'int'> type of each.find('-',0
3d+Medical+Staffing,+LLC = each find
each at end 3d+Medical+Staffing,+LLC
2
<class 'int'> type of each.find('-',0
3d-Medical-Staffing,-LLC = each
3d+Medical+Staffing,+LLC = each find
each at end 3d+Medical+Staffing,+LLC
2
<class 'int'> type of each.find('-',0
3i+International = each find
each at end 3i+International
-1
<class 'int'> type of each.find('-',0
each at end 3M
3
<class 'int'> type of each.find('-',0
4th+Down+Solutions = each find
each at end 4th+Down+Solutions
1
<class 'int'> type of each.find('-',0
A+Dental+Care = each find
each at end A+Dental+Care
2
<class 'int'> type of each.find('-',0
Ab+Staffing+Solutions,+LLC = each find
each at end Ab+Staffing+Solutions,+LLC
2
<class 'int'> type of each.find('-',0
Ab-Staffing-Solutions,-LLC = each
Ab+Staffing+Solutions,+LLC = each find
each at end Ab+Staffing+Solutions,+LLC
-1

In [113]:
company_array_formattedB

['2nd.md',
 '3coast',
 '3d+Medical+Staffing,+LLC',
 '3d+Medical+Staffing,+LLC',
 '3i+International',
 '3M',
 '4th+Down+Solutions',
 'A+Dental+Care',
 'Ab+Staffing+Solutions,+LLC',
 'Ab+Staffing+Solutions,+LLC',
 'Abb',
 'Abbott+Laboratories',
 'Abilities+Unlimited+Inc.',
 'Abm+On++site',
 'Absolute+Physical+Therapy+and+Fitness',
 'Academy+Sports',
 'Academy+Sports+%2B+Outdoors',
 'Acadian+Ambulance',
 'Accc+Insurance+Company',
 'Accenture',
 'Access+Staffing+LLC',
 'Accessible+HR',
 'Accountable+Healthcare+Staffing',
 'Accounting+Principals',
 'Ace+Clinical+Research',
 'Ace+Parking+Management',
 'Activ8+Group',
 'Actuant+Corporation',
 'Acu+of+Texas',
 'Adams+Consulting+Group',
 'Addison+Group',
 'Adecco',
 'Adecco+USA+b5c07591',
 'Adler+Tank+Rentals',
 'Adroit+Partners,+LLC',
 'Advance+Plus+Therapy',
 'Advanced+Search+Group',
 'Advantage+Rent+A+Car',
 'Advantage+Solutions',
 'Advantage+Technical',
 'Advisor+Employee+Services',
 'AECOM',
 'Aequor+Technologies',
 'Aesculap+Inc',
 'Aetna

In [114]:
len(company_array_formattedB)

1917

In [13]:
# creating sub-list for quicker testin
company_array_formatted_sublist = ['Doble-Engineering',
 'Gulfstream-Services-Inc-1',
 'Palm-Health-Resources-1',
 'Rhino-Marketing,-LLC',
 'Genesis-Energy']

company_array_formatted_sublist

['Doble-Engineering',
 'Gulfstream-Services-Inc-1',
 'Palm-Health-Resources-1',
 'Rhino-Marketing,-LLC',
 'Genesis-Energy']

### Two functions that create features in an array, that will need to be put in a feature collection for the geoJSON

In [122]:
# function that takes a datset that is 
def makeFeatureCollectionsFromPlaces(search_term,nameOfSearch):
    dataset_I = requests.get(base_url+search_term+key_pre+key)
    next_page_result = []
    try:
        next_page_token = dataset_I.json()['next_page_token']
        next_page_result = callNextPageResults(nameOfSearch,next_page_token)
    except: 
        pass
    #####if 'next_page_token' in dataset_I:
    #next_page_token = dataset_I.json()['next_page_token']
    #print("next_page_token",next_page_token)
    #callNextPageResults(nameOfSearch,next_page_token)
    #####next_page_results = makeFeatureCollectionsFromPlaces(dataset_I['next_page_token'],nameOfSearch).json()['results']
    ##### #else: 
    ####    next_page = "none needed"
    dataset = dataset_I.json()['results']
    #if 'next_page_token' in dataset_I:
    #    dataset.extend(next_page_results)
    #    print("dataset up top",dataset)
    #else:
    #    next_page = "none needed"
    array_of_features = []
    for each in dataset:
        photos = []
        # centerpoint for coordinates
        #print(each)
        lat = each['geometry']['location']['lat']
        long = each['geometry']['location']['lng']
        # properties
        address = each['formatted_address']
        id = each['id']
        name = each['name']
        # photos is an array
        try:
            photos = each['photos']
        except:
            photos = []
            #print("photos , attribute error but kept going")
        place_id = each['place_id']
        try:
            rating = each['rating']
        except:
            rating = 'NA'
            #print("rating , attribute error but kept going")
        reference = each['reference']
        # types is an array
        types = each['types']
        testPoint = Point((long, lat))
        test2_geoJSOn = Feature(geometry=testPoint, properties={"name": name,"address":address,"id":id,"photos":photos,"place_id":place_id,"rating":rating,"reference":reference,"types":types})
        array_of_features.extend([test2_geoJSOn])
#     new_FeatureCollection = FeatureCollection(array_of_features) 
#     with open('../DATA/'+nameOfSearch+'_test.geojson', 'w') as f:
#         json.dump(new_FeatureCollection, f)
#     return new_FeatureCollection
    print("array_of_features = ",array_of_features," and next_page_result = ",next_page_result)
    print("types: for "+search_term+" type(next_page_result)= ",type(next_page_result)," and type(array_of_features)=  ",type(array_of_features))
    if next_page_result is None or next_page_result==[]:
        fakeVariable = 3
        #print("no second page")
    else:
        #print("next_page_result passed none test and is ",next_page_result)
        array_of_features.extend(next_page_result)
        #print("array_of_features after extension = ",array_of_features)
    #print("2nd statement of array_of_features = ",array_of_features)
    return array_of_features

In [123]:
def callNextPageResults(nameOfSearch,keyForNextPage):
    time.sleep(5)
    #print('next_page_base_url+"&pagetoken="+keyForNextPage+key_pre+key =',next_page_base_url+"pagetoken="+keyForNextPage+key_pre+key)
    dataset_I = requests.get(next_page_base_url+"pagetoken="+keyForNextPage+key_pre+key)
    #print("dataset_I second page full response",dataset_I.json())
    dataset = dataset_I.json()['results']
    array_of_features = []
    for each in dataset:
        photos = []
        # centerpoint for coordinates
        #print(each)
        lat = each['geometry']['location']['lat']
        long = each['geometry']['location']['lng']
        # properties
        address = each['formatted_address']
        id = each['id']
        name = each['name']
        # photos is an array
        try:
            photos = each['photos']
        except:
            photos = []
            #print("photos , attribute error but kept going")
        place_id = each['place_id']
        try:
            rating = each['rating']
        except:
            rating = 'NA'
            #print("rating , attribute error but kept going")
        reference = each['reference']
        # types is an array
        types = each['types']
        testPoint = Point((long, lat))
        test2_geoJSOn = Feature(geometry=testPoint, properties={"name": name,"address":address,"id":id,"photos":photos,"place_id":place_id,"rating":rating,"reference":reference,"types":types})
        array_of_features.extend([test2_geoJSOn])
    #new_FeatureCollection = FeatureCollection(array_of_features) 
    #with open('../DATA/'+nameOfSearch+'_2ndPage_test.geojson', 'w') as f:
    #    json.dump(new_FeatureCollection, f)
    #print("second page results = ",new_FeatureCollection)
    #return new_FeatureCollection
    return array_of_features

### A function that interates through the array of companies and collects the arrays of geographic features for the geojson.

In [124]:
def createGeoJSONfromList(cp_array_formatted):
    array_of_all_features = []
    for each in cp_array_formatted:
        #try:
        features_1 = makeFeatureCollectionsFromPlaces(each+"+Houston+Texas","bigTest1")
        #print("features for a company name =",features_1)
        if array_of_all_features == [] or array_of_all_features is None:
            array_of_all_features = features_1
        else:
            if features_1 is None or features_1 == []:
                pass
            else:
                array_of_all_features.extend(features_1)
        #except:
        #    print("error in except",error)
        
        #print("array_of_all_features = ",array_of_all_features)
    
    return array_of_all_features

In [56]:
company_array_formatted_sublist

['Doble-Engineering',
 'Gulfstream-Services-Inc-1',
 'Palm-Health-Resources-1',
 'Rhino-Marketing,-LLC',
 'Genesis-Energy']

In [120]:
company_array_formattedC_atob =['2nd.md',
 '3coast',
 '3d+Medical+Staffing,+LLC',
 '3d+Medical+Staffing,+LLC',
 '3i+International',
 '3M',
 '4th+Down+Solutions',
 'A+Dental+Care',
 'Ab+Staffing+Solutions,+LLC',
 'Ab+Staffing+Solutions,+LLC',
 'Abb',
 'Abbott+Laboratories',
 'Abilities+Unlimited+Inc.',
 'Abm+On++site',
 'Absolute+Physical+Therapy+and+Fitness',
 'Academy+Sports',
 'Academy+Sports+%2B+Outdoors',
 'Acadian+Ambulance',
 'Accc+Insurance+Company',
 'Accenture',
 'Access+Staffing+LLC',
 'Accessible+HR',
 'Accountable+Healthcare+Staffing',
 'Accounting+Principals',
 'Ace+Clinical+Research',
 'Ace+Parking+Management',
 'Activ8+Group',
 'Actuant+Corporation',
 'Acu+of+Texas',
 'Adams+Consulting+Group',
 'Addison+Group',
 'Adecco',
 'Adecco+USA+b5c07591',
 'Adler+Tank+Rentals',
 'Adroit+Partners,+LLC',
 'Advance+Plus+Therapy',
 'Advanced+Search+Group',
 'Advantage+Rent+A+Car',
 'Advantage+Solutions',
 'Advantage+Technical',
 'Advisor+Employee+Services',
 'AECOM',
 'Aequor+Technologies',
 'Aesculap+Inc',
 'Aetna',
 'Affiliated+Engineers,+Inc.',
 'Affinity',
 'Afglobal+Corporation',
 'Aftermath+Services',
 'Afton+Chemical',
 'Aggreko,+LLC',
 'Aggreko,+LLC',
 'Agile+Mind',
 'Agile+Premier',
 'Agilent',
 'Agl+Resources',
 'Agrilectric+Power',
 'Aids+Healthcare+Foundation',
 'AIG',
 'Air+Liquide',
 'Air+Products',
 'Airgas',
 'Ajilon',
 'Akima,+LLC',
 'Akima,+LLC',
 'Akzonobel',
 'Alamo+Pharma+Services',
 'Aldine+Westfield+Animal+Hospital',
 'Alert+Logic',
 'Alion+Science+and+Technology',
 'Alios+Medical+Staffing',
 'Alixpartners',
 'All+Pets+Animal+Hospital',
 'Allergy+&+Ent+Associates',
 'Allergy+Ent+Associates',
 'Allianceone',
 'Alliant+Insurance+Services',
 'Alliantgroup',
 'Allied+Lube+Texas+Lp',
 'Allied+Search+Partners',
 'Alpheus+Enterprise',
 'Als+Global',
 'Alsac',
 'Altisource',
 'Altus+Baytown+Hospital',
 'Altus+Group',
 'Altus+Hospice',
 'Alvarez+&+Marsal',
 'Alvarez+&+Marsal',
 "Alzheimer's+Association",
 'Alzheimer%27s+Association',
 'Amag+Pharmaceuticals,+Inc',
 'Amazon.com',
 'Amazon.com?campaignid=femp&from=femp',
 'Ambassadors+For+Christ+Youth+Ministries,+Inc',
 'Ambassadors+For+Christ+Youth+Ministries,+Inc',
 'Ambs+Chemical+Search',
 'Amd+Asset+Holdings,+LLC',
 'Amd+Asset+Holdings,+LLC',
 'Amegy+Bank',
 'America+Needs+You',
 'American+Heart+Association',
 'American+Homes+4+Rent',
 'American+Midstream+GP+LLC',
 'American+National+Family+of+Companies',
 'American+Traveler',
 'Americorps',
 'Amerisourcebergen',
 'Amerit+Fleet+Solutions',
 'Amot',
 'Amspec+Services',
 'Analytical+Sensors',
 'Andersen+Tax',
 'Anixter',
 'Ansys',
 'Anton+Paar',
 'Aok+Medical+Center',
 'Aon',
 'Apache+Corporation',
 'Apex',
 'Apex+Companies,+LLC',
 'Apex+Companies,+LLC',
 'Apex+Life+Sciences',
 'Apocell',
 'Appirio',
 'Applied+Diagnostics',
 'Applied+Lab+Automation+Corp.',
 'Applied+Optoelectronics+Inc.',
 'Applied+Physics+Systems',
 'Applied+Resource+Group',
 'Apr+Consulting,+Inc.',
 'Apria+Healthcare',
 'Aqua++seas+Seafood+Market',
 'Aquent+9',
 'Aramark',
 'Arcadis',
 'Archrock',
 'Ares+Corporation',
 'Argus+Media',
 'Arkema+Americas',
 'Armstrong+Flooring',
 'Array+Biopharma',
 'Ars',
 'Ars%2Frescue+Rooter',
 'Art+League+Houston',
 'Arth+Consulting',
 'Arup',
 'Asahi+Kasei',
 'Ascend+Performance+Materials',
 'Ascendo+Resources',
 'Ascension+Episcopal+School',
 'Ascent+Business+Solutions',
 'Ascent+Services+Group',
 'Asg+Security',
 'Aspen+Technology',
 'Associated+Brigham+Contractors',
 'Associated+Credit+Union+of+Texas',
 'Associated+Testing+Laboratories',
 'Assure+Oilfield+Testing,+Inc.',
 'Astellas+Pharmaceuticals',
 'Astrazeneca',
 'Atria+Senior+Living+Group',
 'Atria+Sugar+Land',
 'Atria+Westchase',
 'Atser',
 'Attorney+Job+Finder',
 'Austin+Industries',
 'Autolab+of+Gleannloch+Farms',
 'Automationtechies',
 'Autonation',
 'Autumn+Leaves',
 'Autumn+Leaves+Assisted+Living',
 'Avangard+Innovative',
 'Avella+Specialty+Pharmacy',
 'Avispa+Technology',
 'Axens+North+America',
 'Axiem+International',
 'Axis+Medical+Staffing',
 'Axius+Technologies',
 'Azule',
 'Azuma+Leasing',
 'Babcock+&+Wilcox',
 'Babcock+&+Wilcox',
 'Babel+Therapy,+Pllc',
 'Babel+Therapy,+Pllc',
 'Baker+Botts+L.l.p.',
 'Baker+Hughes+Incorporated',
 'Bakercorp',
 'Ballistix',
 'Banfield+Pet+Hospital',
 'Bank+of+America',
 'Banyan+Equity+Management,+LLC',
 'Banyan+Equity+Management,+LLC',
 'Barker+Law+Firm+PC',
 'Barnhart+Crane+&+Rigging',
 'Barnhart+Crane+&+Rigging',
 'Barrios+Technology',
 'Barry++wehmiller+Design+Group',
 'Barton+Associates',
 'Basf',
 'Bashen+Corporation',
 'Basintek+LLC',
 'Bay++porte+Animal+Hospital',
 'Bay+Area+Houston+Endoscopy',
 'Bay+Area+Regional+Medical+Center',
 'Baylor+College+of+Medicine',
 'Baylor+College+of+Medicine+(bcm)',
 'Baylor+College+of+Medicine+%28bcm%29',
 'Baylor+Medicine',
 'Baylor+Miraca+Genetics+Laboratories+LLC',
 'Bayshore+Medical+Center+++Pasadena',
 'Bb&T',
 'Bb&T',
 'Bcfs',
 'Bdo',
 'Beam',
 'Bechtel',
 'Beckman+Coulter,+Inc.',
 'Becton+Dickinson+&+Company',
 'Becton+Dickinson+&+Company',
 'Belcan',
 'Belfiore',
 'Bell+Tech+Home+Health+Care',
 'Bellicum+Pharmaceuticals',
 'Best+Buy',
 'Best+Friends+Animal+Society',
 'Bestcare+Laboratory',
 'Beta+Academy',
 'Bg+Staffing',
 'Bh+Management+Services,+LLC',
 'Bh+Management+Services,+LLC',
 'Bh+Management+Services,+LLC',
 'Bh+Management+Services,+LLC',
 'Bhi+Energy',
 'Bhp+Billiton',
 'Big+Delta+Systems',
 'Bileddo+Associates',
 'Bilingual+Psychological+Associates+PC',
 'Bilingual+Therapies',
 'Bio++reference+Laboratories,+Inc.',
 'Bionical+LLC',
 'Bioscrip,+Inc.',
 'Black+&+Veatch',
 'Black+&+Veatch',
 'Blackrock+Talent',
 'Blinds.com',
 'Blu+Phoenix+Fitness',
 'Blue+Quality+Services',
 'Blue+Royal+Staffing',
 'Bluepearl+Veterinary+Partners',
 'Bmc+Software',
 'Bmo+Financial+Group',
 'Bny+Mellon',
 'Boardwalk+Pipeline+Partners',
 'Bolttech+Mannings',
 'Booz+Allen+Hamilton',
 'Boston+Scientific',
 'Boxer+Property',
 'Boy+Scouts+of+America',
 'BP',
 'Brazosport+College',
 'Bridgepoint+Consulting',
 'Bridgestone+Americas',
 'Brighter+Day+Health',
 'Brightstar+Care+of+Friendswood',
 'Brightview',
 'Brio+Resource+Group',
 'Broadspectrum',
 'Brookdale',
 'Brown+and+Caldwell',
 "Buc++ee's",
 'Buc++ee%27s',
 'Buckeye+Partners',
 'Bureau+Veritas',
 'Burgess+&+Niple',
 'Burgess+&+Niple',
 'Burnett+Specialists',
 'Burnett+Specialists+Choice+Specialists',
 'Burns+&+Mcdonnell',
 'Burns+&+Wilcox',
 'Burns+&+Mcdonnell',
 'Burns+&+Wilcox',
 'Busch+LLC']

In [125]:
company_array_formattedC_atob

['2nd.md',
 '3coast',
 '3d+Medical+Staffing,+LLC',
 '3d+Medical+Staffing,+LLC',
 '3i+International',
 '3M',
 '4th+Down+Solutions',
 'A+Dental+Care',
 'Ab+Staffing+Solutions,+LLC',
 'Ab+Staffing+Solutions,+LLC',
 'Abb',
 'Abbott+Laboratories',
 'Abilities+Unlimited+Inc.',
 'Abm+On++site',
 'Absolute+Physical+Therapy+and+Fitness',
 'Academy+Sports',
 'Academy+Sports+%2B+Outdoors',
 'Acadian+Ambulance',
 'Accc+Insurance+Company',
 'Accenture',
 'Access+Staffing+LLC',
 'Accessible+HR',
 'Accountable+Healthcare+Staffing',
 'Accounting+Principals',
 'Ace+Clinical+Research',
 'Ace+Parking+Management',
 'Activ8+Group',
 'Actuant+Corporation',
 'Acu+of+Texas',
 'Adams+Consulting+Group',
 'Addison+Group',
 'Adecco',
 'Adecco+USA+b5c07591',
 'Adler+Tank+Rentals',
 'Adroit+Partners,+LLC',
 'Advance+Plus+Therapy',
 'Advanced+Search+Group',
 'Advantage+Rent+A+Car',
 'Advantage+Solutions',
 'Advantage+Technical',
 'Advisor+Employee+Services',
 'AECOM',
 'Aequor+Technologies',
 'Aesculap+Inc',
 'Aetna

In [126]:
#company_array_formattedB

#### make a new feature collection of all the results from the google API based on a list of companies from the web scraping
new_FeatureCollectionBig_C = FeatureCollection(createGeoJSONfromList(company_array_formattedC_atob))
#new_FeatureCollectionBigA
#### converting feature collection to geoJSON and saving as a file
convertFeatureCollectionToGeoJSON(new_FeatureCollectionBig_C,'../DATA/Big_company_list_A-B_v6big_test.geojson')

array_of_features =  [{"geometry": {"coordinates": [-95.46029370000001, 29.7531235], "type": "Point"}, "properties": {"address": "1300 Post Oak Blvd #725, Houston, TX 77056, United States", "id": "29c1263f5053b5d041ce4cb435f91f31c81dc285", "name": "2ndMD", "photos": [{"height": 798, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/106166440780103249972/photos\">2ndMD</a>"], "photo_reference": "CoQBdwAAAJHecjrk386coz06OfT6DJs0Mut2CvzvHOpZV8z6yuJ_NFCee-25sU8Znkz-GqFYSktkOlEG1bM8YrUl0G5F6NK8iicgrJNjafyF4wpfzaaZ85aYGK-bsZqTMOGhA2QmouHjjDxpVZKm1MsjNlh_AGnMKzwFGqczmnJeiYcOg7ROEhA1pPj12uyzA-5bO1Rg6fuQGhQYKgcTHEi_eOxfyg0VmAPbjNUyvg", "width": 800}], "place_id": "ChIJn-IydsTDQIYRSVSPAA63ivY", "rating": "NA", "reference": "CmRSAAAAUaNHfl57DRfGKIGfjWSaaAVZNHltKMyntqoA4NBoQ9-6xg62oekNVzd8AcumCpsS7C2chwpK5bPTw1frWxW71ag3Z-OEFKci-byNcY20qDkMY5jqUB_GxluKwIT_-R-YEhBVfXnpYAZwEb7YZofUNj7TGhSqj_n8Nd5utxa204EcM_333gLZKQ", "types": ["hospital", "point_of_interest", "establishment"]}, "

NameError: name 'convertFeatureCollectionToGeoJSON' is not defined

In [127]:
new_FeatureCollectionBig_C

{"features": [{"geometry": {"coordinates": [-95.46029370000001, 29.7531235], "type": "Point"}, "properties": {"address": "1300 Post Oak Blvd #725, Houston, TX 77056, United States", "id": "29c1263f5053b5d041ce4cb435f91f31c81dc285", "name": "2ndMD", "photos": [{"height": 798, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/106166440780103249972/photos\">2ndMD</a>"], "photo_reference": "CoQBdwAAAJHecjrk386coz06OfT6DJs0Mut2CvzvHOpZV8z6yuJ_NFCee-25sU8Znkz-GqFYSktkOlEG1bM8YrUl0G5F6NK8iicgrJNjafyF4wpfzaaZ85aYGK-bsZqTMOGhA2QmouHjjDxpVZKm1MsjNlh_AGnMKzwFGqczmnJeiYcOg7ROEhA1pPj12uyzA-5bO1Rg6fuQGhQYKgcTHEi_eOxfyg0VmAPbjNUyvg", "width": 800}], "place_id": "ChIJn-IydsTDQIYRSVSPAA63ivY", "rating": "NA", "reference": "CmRSAAAAUaNHfl57DRfGKIGfjWSaaAVZNHltKMyntqoA4NBoQ9-6xg62oekNVzd8AcumCpsS7C2chwpK5bPTw1frWxW71ag3Z-OEFKci-byNcY20qDkMY5jqUB_GxluKwIT_-R-YEhBVfXnpYAZwEb7YZofUNj7TGhSqj_n8Nd5utxa204EcM_333gLZKQ", "types": ["hospital", "point_of_interest", "establishment"]}, "type": "

In [128]:
# convert feature collection to geoJSON
def convertFeatureCollectionToGeoJSON(featCollection,fileNamePath):
    with open(fileNamePath, 'w') as f:
        json.dump(featCollection, f)


In [129]:
#### converting feature collection to geoJSON and saving as a file
convertFeatureCollectionToGeoJSON(new_FeatureCollectionBig_C,'../DATA/Big_company_list_A-B_v6big_test.geojson')

In [ ]:
createGeoJSONfromList(cp_array_formatted)

In [ ]:
# the one above here is what I"m waiting to finish 

In [ ]:
convertFeatureCollectionToGeoJSON(new_FeatureCollectionBigA,'../DATA/Big_company_list_v5big_test.geojson')

In [39]:
new_FeatureCollection = FeatureCollection(createGeoJSONfromList(company_array_formatted_sublist))
new_FeatureCollection

{'formatted_address': '1000 S Loop W #100, Houston, TX 77054, United States', 'reference': 'CmRRAAAAdzUGcA9Da7y8gd6WmH9SpxEcVvGJBz8d2xVPD7w7ApbkvMCgXyznyUeAD1lFBuuAp8DhKPIQQQAq-76D6i5Q84A_ETV9UjanVo1FSDWJSbk0e2nPvJziuZHqzQ6kYZABEhDwWcxlQAdSQIUT1_qhj-W9GhRXiNm-ZOALuxSSfE8yCOMVlME1fA', 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'geometry': {'viewport': {'southwest': {'lng': -95.3918668302915, 'lat': 29.6798140697085}, 'northeast': {'lng': -95.3891688697085, 'lat': 29.68251203029149}}, 'location': {'lng': -95.39051959999999, 'lat': 29.6812243}}, 'name': 'Doble', 'id': 'd7d864991744cf9af94c72de181f4bf267524e01', 'types': ['point_of_interest', 'establishment'], 'place_id': 'ChIJBwBYRv6_QIYRatQETbtlQXU'}
{'formatted_address': '12319 Lynda Dr, Houston, TX 77038, United States', 'types': ['electrician', 'point_of_interest', 'establishment'], 'reference': 'CmRRAAAAec4cvBVkb7PjK1NDvl-zDEsus9M49JSlYjjdDTEvuscfPG6V9JYwmP-sO7xVG62NBJTpg3OuhK7f6_JNhlul3taph0

{"features": [{"geometry": {"coordinates": [-95.39051959999999, 29.6812243], "type": "Point"}, "properties": {"address": "1000 S Loop W #100, Houston, TX 77054, United States", "id": "d7d864991744cf9af94c72de181f4bf267524e01", "name": "Doble", "photos": [], "place_id": "ChIJBwBYRv6_QIYRatQETbtlQXU", "rating": "NA", "reference": "CmRRAAAAdzUGcA9Da7y8gd6WmH9SpxEcVvGJBz8d2xVPD7w7ApbkvMCgXyznyUeAD1lFBuuAp8DhKPIQQQAq-76D6i5Q84A_ETV9UjanVo1FSDWJSbk0e2nPvJziuZHqzQ6kYZABEhDwWcxlQAdSQIUT1_qhj-W9GhRXiNm-ZOALuxSSfE8yCOMVlME1fA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4510504, 29.9099788], "type": "Point"}, "properties": {"address": "12319 Lynda Dr, Houston, TX 77038, United States", "id": "b602b2c55fe6130ab68dbfd31db4382763879b0c", "name": "Doble E Electric", "photos": [{"height": 1224, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/110775628567700969334/photos\">Doble E Electric</a>"], "photo_reference": "CoQ

In [40]:
new_FeatureCollection

{"features": [{"geometry": {"coordinates": [-95.39051959999999, 29.6812243], "type": "Point"}, "properties": {"address": "1000 S Loop W #100, Houston, TX 77054, United States", "id": "d7d864991744cf9af94c72de181f4bf267524e01", "name": "Doble", "photos": [], "place_id": "ChIJBwBYRv6_QIYRatQETbtlQXU", "rating": "NA", "reference": "CmRRAAAAdzUGcA9Da7y8gd6WmH9SpxEcVvGJBz8d2xVPD7w7ApbkvMCgXyznyUeAD1lFBuuAp8DhKPIQQQAq-76D6i5Q84A_ETV9UjanVo1FSDWJSbk0e2nPvJziuZHqzQ6kYZABEhDwWcxlQAdSQIUT1_qhj-W9GhRXiNm-ZOALuxSSfE8yCOMVlME1fA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4510504, 29.9099788], "type": "Point"}, "properties": {"address": "12319 Lynda Dr, Houston, TX 77038, United States", "id": "b602b2c55fe6130ab68dbfd31db4382763879b0c", "name": "Doble E Electric", "photos": [{"height": 1224, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/110775628567700969334/photos\">Doble E Electric</a>"], "photo_reference": "CoQ

In [42]:
convertFeatureCollectionToGeoJSON(new_FeatureCollection,'../DATA/Big_company_list_v3small_test.geojson')

### Another test to make sure the next page functionality works - this one also includes Shell, which should have a lot of gas stations in Houston.

In [82]:
# creating sub-list for quicker testin
company_array_formatted_sublist2 = ['Genesis-Energy','Shell']

company_array_formatted_sublist2

['Genesis-Energy', 'Shell']

In [108]:
new_FeatureCollection2 = FeatureCollection(createGeoJSONfromList(company_array_formatted_sublist2))
new_FeatureCollection2

{'formatted_address': '919 Milam St #2100, Houston, TX 77002, United States', 'rating': 1, 'reference': 'CmRSAAAATubIDf7z1sl9KG38m25lAsmOkK4yx6tZ0cbbCdzqtJLkydLAi9ALPz-qcADsBFC-dY-OfmLr-Z3NSEt7c-kxQwEEB_Q8B0nIeSvL6eZxX0fciqS8GxtVV8mygYdztAtlEhAbWmQHfn8GOuQV8hzO7hsYGhS1rJHHoMEl8N-PP71ib3FXB6LACQ', 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'geometry': {'viewport': {'southwest': {'lng': -95.36747293029151, 'lat': 29.7568735697085}, 'northeast': {'lng': -95.36477496970849, 'lat': 29.7595715302915}}, 'location': {'lng': -95.3659038, 'lat': 29.75805819999999}}, 'name': 'Genesis Energy, LP', 'id': '7dbf977f8ebe642724c868999336f79ab8361447', 'types': ['point_of_interest', 'establishment'], 'place_id': 'ChIJfy2iBDm_QIYRJ3VGXRbTSYg'}
array_of_features =  [{"geometry": {"coordinates": [-95.3659038, 29.75805819999999], "type": "Point"}, "properties": {"address": "919 Milam St #2100, Houston, TX 77002, United States", "id": "7dbf977f8ebe642724c868999336f79

{"features": [{"geometry": {"coordinates": [-95.3659038, 29.75805819999999], "type": "Point"}, "properties": {"address": "919 Milam St #2100, Houston, TX 77002, United States", "id": "7dbf977f8ebe642724c868999336f79ab8361447", "name": "Genesis Energy, LP", "photos": [], "place_id": "ChIJfy2iBDm_QIYRJ3VGXRbTSYg", "rating": 1, "reference": "CmRSAAAATubIDf7z1sl9KG38m25lAsmOkK4yx6tZ0cbbCdzqtJLkydLAi9ALPz-qcADsBFC-dY-OfmLr-Z3NSEt7c-kxQwEEB_Q8B0nIeSvL6eZxX0fciqS8GxtVV8mygYdztAtlEhAbWmQHfn8GOuQV8hzO7hsYGhS1rJHHoMEl8N-PP71ib3FXB6LACQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.372894, 29.747668], "type": "Point"}, "properties": {"address": "2220 Main St, Houston, TX 77002, United States", "id": "a008b524f02cd1beaed229f80d06f23a73b68bb7", "name": "Shell", "photos": [{"height": 2183, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/105976331058990070645/photos\">WOLFGANG DEMINO</a>"], "photo_reference": "CoQBdwAAA

In [109]:
convertFeatureCollectionToGeoJSON(new_FeatureCollection2,'../DATA/Big_company_list_v4small_PlusShell_test.geojson')

In [ ]:
convertFeatureCollectionToGeoJSON(new_FeatureCollection,'../DATA/Big_company_list_v3small_test.geojson')